In [ ]:
# Execute this cell, only if you didn't add latent_3d_points to your $PYTHONPATH
import sys
sys.path.append("../")
from src import *
from external import *

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os.path as osp

from src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from src.autoencoder import Configuration as Conf

from src.point_net_ae import PointNetAutoEncoder

from src.in_out import snc_category_to_synth_id, Poi

from external.general_tools.in_out.basics import create_dir

from external.general_tools.notebook.tf import reset_tf_graph

In [33]:
# import os.path as osp

# from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

# from latent_3d_points.src.autoencoder import Configuration as Conf
# from latent_3d_points.src.point_net_ae import PointNetAutoEncoder
# from latent_3d_points.src.in_out import snc_category_to_synth_id

# from latent_3d_points.external.general_tools.in_out.basics import create_dir
# from latent_3d_points.external.general_tools.notebook.tf import reset_tf_graph

In [34]:
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet

# from tf_lab.in_out.basics import Data_Splitter
# from tf_lab.data_sets.shape_net import pc_loader as snc_loader
# from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats

# TODO : DATA LOADING

Define Basic Parameters

In [26]:
top_data_dir = '../data/'
experiment_name = 'single_class_ae'
n_pc_points = 2048
class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()
bneck_size = 128
ae_loss = 'emd'

Give me the class name (e.g. "chair"): chair


Use Default Training Parameters

{'batch_size': 50,
 'denoising': False,                   # By default our AE is not denoising.
 'learning_rate': 0.0005,
 'loss_display_step': 1,
 'saver_step': 10,
 'training_epochs': 500,
 'z_rotate': False}


In [38]:
train_dir = create_dir(osp.join(top_data_dir, experiment_name))
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)

In [39]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5
conf.save(osp.join(train_dir, 'configuration'))

Build AE Model.

In [33]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

Building Encoder
encoder_conv_layer_0 conv params =  256 bnorm params =  128
Tensor("single_class_ae_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_1 conv params =  8320 bnorm params =  256
Tensor("single_class_ae_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_2 conv params =  16512 bnorm params =  256
Tensor("single_class_ae_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  33024 bnorm params =  512
Tensor("single_class_ae_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  32896 bnorm params =  256
Tensor("single_class_ae_2/Relu_4:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

Tensor("single_class_ae_2/Max:0", shape=(?, 128), dtype=float32)
Building Decoder
decoder_fc_0 FC params =  33024 Tensor("single_class_ae_2/Relu_5:0", shape=(?, 256), dtype=float32)
output size: 256 

In [ ]:
# Start training
buf_size = 1 # flush each line
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(in_data['train'], conf, log_file=fout, held_out_data=in_data['val'])
fout.close()